[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HashimHilal-QUT/LLM/blob/main/AI_QuizMaster/Quiz_Master_from_Text_Book.ipynb)

# Introduction

This Notebook will run on Google Colab and uses Google AI engine to create a Quiz Master for students to practice, using Text Book source as dataset

In [ ]:
# @title Setup Environment
!pip install python-docx
from google.colab import ai
from docx import Document
import requests
import io
import base64
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("✅ Environment ready.")

## Initialize Secure Data Connection

Current Source Datasets :

1. Fundamentals of Python : First Programs Paperback by Kenneth Lambert (Author)
2. CompTIA Security+ Guide to Network Security Fundamentals, 8e by ​Mark Kiampa (Author)


In [ ]:
# Textbook Dataset Selection
options = ['Option 1: Fundamentals of Python', 'Option 2: CompTIA Security+ Guide']
dropdown = widgets.Dropdown(options=options, value=options[0], description='Select Dataset:')

def on_change(change):
    global UserSelect
    UserSelect = change['new']

dropdown.observe(on_change, names='value')

# Initialize UserSelect with default value
UserSelect = dropdown.value

display(dropdown)

In [ ]:
# Source Dataset API Key

if UserSelect == 'Option 1: Fundamentals of Python':
    _m = "aHR0cHM6Ly9xdXR0ZXN0ZGF0YWFpMTAyLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kYXRhc2V0L1F1aXpNYXN0ZXIuZG9jeA=="
elif UserSelect == 'Option 2: CompTIA Security+ Guide':
    _m = "aHR0cHM6Ly9xdXR0ZXN0ZGF0YWFpMTAyLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kYXRhc2V0L1F1aXpCYW5rLmRvY3g="

In [ ]:
# Code to test connection and load Question Bank
try:
    
    _u = base64.b64decode(_m).decode('utf-8')
    response = requests.get(_u)
    response.raise_for_status()
    
    doc = Document(io.BytesIO(response.content))
    question_bank = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
    
    
    clear_output()
    print("✅ Question Bank loaded ")

except Exception as e:
    clear_output()
    print("❌ Connection Error. Please check your internet or repository access.")

### Start Interactive AI powered Quiz 

In [ ]:

# 1. Custom CSS to force larger font in text boxes and buttons
display(HTML("""
<style>
    .widget-text input { font-size: 16px !important; line-height: 1.5 !important; }
    .widget-output { font-size: 16px !important; }
    .widget-button { font-weight: bold !important; }
</style>
"""))

# Initialize conversation state
state = {
    'transcript': f"""SYSTEM: You are a Strict Quiz Master. 
    SOURCE MATERIAL: {question_bank}
    STRICT RULES:
    1. Only use questions from SOURCE MATERIAL.
    2. Answers are in Bold text or have the word "Answer" before them.
    3. Provide feedback if the answer is correct and then move on to the next question in ONE message.
    4. No conversational filler or follow-up questions.
    5. Provide 'FINAL GRADE REPORT' at the end or on exit.
    """,
    'is_finished': False
}

# --- UI Elements with Larger Styling ---
output_area = widgets.Output(layout={
    'border': '2px solid #4A90E2', 
    'padding': '15px', 
    'height': '500px', 
    'overflow_y': 'scroll',
    'margin': '0 0 10px 0'
})

text_input = widgets.Text(
    placeholder='Type your answer here...', 
    layout={'width': '70%', 'height': '45px'}
)

send_button = widgets.Button(
    description='Send Answer', 
    button_style='primary',
    layout={'width': '15%', 'height': '45px'}
)

exit_button = widgets.Button(
    description='Exit & Grade', 
    button_style='danger',
    layout={'width': '15%', 'height': '45px'}
)

def get_ai_response(user_text):
    state['transcript'] += f"\nStudent: {user_text}\nAI:"
    raw_response = ai.generate_text(state['transcript'])
    state['transcript'] += f" {raw_response}"
    return raw_response.replace("**", "")

def handle_send(b):
    if state['is_finished']: return
    user_val = text_input.value.strip()
    if not user_val: return
    
    with output_area:
        print(f"➜ You: {user_val}")
        text_input.value = "" 
        response = get_ai_response(user_val)
        print(f"🤖 AI: {response}\n" + "━"*40)
        
        if "FINAL GRADE REPORT" in response:
            state['is_finished'] = True
            text_input.disabled = True

def handle_exit(b):
    if state['is_finished']: return
    with output_area:
        print("\n🛑 Ending Quiz Early...")
        response = get_ai_response("The student clicked EXIT. Stop the quiz and provide the FINAL GRADE REPORT now.")
        print(f"🤖 AI: {response}")
        state['is_finished'] = True
        text_input.disabled = True

send_button.on_click(handle_send)
text_input.on_submit(handle_send)
exit_button.on_click(handle_exit)

# --- Launch UI ---
display(output_area)
display(widgets.HBox([text_input, send_button, exit_button]))

with output_area:
    first_msg = get_ai_response("Start the quiz now by asking Question 1 from the bank.")
    print(f"🤖 AI: {first_msg}\n" + "━"*80)

### Note: 
Since the Notebook uses ipywidgets, please clear all Cell outputs before saving to GitHub, otherwise it will not preview correctly in Github.

You will see error "There was an error rendering your Notebook" , This error occurs because GitHub's built-in notebook viewer is trying to render the IPython Widgets (the buttons and text boxes we created) but cannot find the "widget state" in the notebook's underlying JSON metadata.

GitHub (and most static viewers) cannot render interactive widgets because they require a live Python kernel (like the one running in Google Colab) to function.